<a href="https://colab.research.google.com/github/SAHIL9581/w2w/blob/main/W2W_WNB_INFERENCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#@title Step 1: Upload Raw .las Data ZIP
from google.colab import files
import os

print("--> Please upload the ZIP file containing the folder of .las files.")
uploaded = files.upload()

if not uploaded:
  print("\n⚠️ Upload cancelled. Please run the cell again.")
else:
  zip_filename = list(uploaded.keys())[0]
  print(f"\n✅ '{zip_filename}' uploaded successfully.")
  # Unzip the file. This should create the 'Force_2020...' folder.
  !unzip -q -o "{zip_filename}" -d /content/
  os.remove(zip_filename)
  print("✅ Raw data unzipped.")

--> Please upload the ZIP file containing the folder of .las files.


Saving train.zip to train.zip

✅ 'train.zip' uploaded successfully.
✅ Raw data unzipped.


In [12]:
#@title Step 2: Upload Trained Model and Scaler
from google.colab import files
import os

print("--> Please upload your 'final_model.pt' and 'StandardScaler.bin' files.")

# Create directories to store the files
os.makedirs("/content/trained_models/boundary_detector", exist_ok=True)
os.makedirs("/content/artifacts", exist_ok=True)

# Upload model
print("\nUploading model...")
uploaded_model = files.upload()
if uploaded_model:
    model_name = list(uploaded_model.keys())[0]
    os.rename(f"/content/{model_name}", f"/content/trained_models/boundary_detector/{model_name}")
    print(f"✅ Model '{model_name}' moved to the correct directory.")

# Upload scaler
print("\nUploading scaler...")
uploaded_scaler = files.upload()
if uploaded_scaler:
    scaler_name = list(uploaded_scaler.keys())[0]
    os.rename(f"/content/{scaler_name}", f"/content/artifacts/{scaler_name}")
    print(f"✅ Scaler '{scaler_name}' moved to the correct directory.")

--> Please upload your 'final_model.pt' and 'StandardScaler.bin' files.

Uploading model...



Uploading scaler...


In [13]:
#@title Step 3: Setup Environment and Configuration
# --- 1. Install & Import Libraries ---
print("--> Installing libraries...")
!pip install wandb torch lasio scikit-learn pandas matplotlib joblib -q
import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import wandb
from joblib import load
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import lasio
print("✅ Libraries ready.")

# --- 2. Define Configuration ---
# This config now points to the folder of .las files
config = {
    "paths": {
        "raw_las_folder": "/content/Force_2020_all_wells_train_test_blind_hidden_final/",
        "std_scaler_path": "/content/artifacts/StandardScaler.bin",
        "final_model_path": "/content/trained_models/boundary_detector/final_model.pt"
    },
    "wandb": {
        "project": "W2W_Matcher_Pipeline_Notebook_Inference",
        "entity": None,
    },
    "finetuning": {
        "model_params": {
            "in_channels": 9, # IMPORTANT: Must match the features model was trained on
            "patch_height": 700, "act_name": "prelu",
            "hidden_dim": 256, "num_queries": 100,
            "num_heads": 8, "dropout": 0.1,
            "num_transformers": 6, "output_size": 3
        }
    },
    "inference": {
      "prediction_score_threshold": 0.6
    }
}
print("✅ Configuration loaded.")

--> Installing libraries...
✅ Libraries ready.
✅ Configuration loaded.


In [14]:
#@title Step 4: Define Model Architectures
def get_activation(name): return nn.PReLU() if name == 'prelu' else nn.ReLU() if name == 'relu' else nn.GELU()
class Block1D(nn.Module):
    def __init__(self, i, o, s=2, k=3, a='prelu'): super().__init__(); self.b = nn.Sequential(nn.Conv1d(i,o,k,s,padding=k//2), nn.BatchNorm1d(o), get_activation(a), nn.Conv1d(o,o,k,1,padding=k//2), nn.BatchNorm1d(o), get_activation(a))
    def forward(self, x): return self.b(x)
class UNetEncoder1D(nn.Module):
    def __init__(self, i, a='prelu'): super().__init__(); self.s=Block1D(i,32,s=1,a=a); self.e1=Block1D(32,64,s=2,a=a); self.e2=Block1D(64,128,s=2,a=a); self.e3=Block1D(128,256,s=2,a=a); self.m=Block1D(256,512,s=2,a=a)
    def forward(self, x): x=x.squeeze(1).permute(0,2,1); s1=self.s(x); s2=self.e1(s1); s3=self.e2(s2); s4=self.e3(s3); return self.m(s4)
class Project(nn.Module):
    def __init__(self,i,o): super().__init__(); self.l=nn.Linear(i,o)
    def forward(self,x): return self.l(x.flatten(1))
class Query(nn.Module):
    def __init__(self,s,d): super().__init__(); self.q=nn.Parameter(torch.randn(1,s,d))
    def forward(self,x): return self.q.repeat(x.shape[0],1,1)
class Transformer(nn.Module):
    def __init__(self,i,n,d): super().__init__(); self.t=nn.TransformerEncoderLayer(d_model=i,nhead=n,dropout=d,batch_first=True,dim_feedforward=i*4)
    def forward(self,q,c): return self.t(q)
class W2WTransformerModel(nn.Module):
    def __init__(self,c):
        super().__init__(); p=c['finetuning']['model_params']; self.e=UNetEncoder1D(p['in_channels'],p['act_name'])
        with torch.no_grad(): p_in=self.e(torch.randn(1,1,p['patch_height'],p['in_channels'])).flatten(1).shape[1]
        self.p=Project(p_in,p['hidden_dim']); self.q=Query(p['num_queries'],p['hidden_dim']); self.t=nn.ModuleList([Transformer(p['hidden_dim'],p['num_heads'],p['dropout']) for _ in range(p['num_transformers'])]); self.f=nn.Sequential(nn.Linear(p['hidden_dim'],p['output_size']),get_activation(p['act_name']),nn.LayerNorm(p['output_size']))
    def forward(self,img): seq=self.p(self.e(img)).unsqueeze(1); q=self.q(seq); [q:=t(q,seq) for t in self.t]; return self.f(q)
print("✅ Model architectures defined.")

✅ Model architectures defined.


In [15]:
#@title Step 5: Define Inference and Plotting Functions
def predict_layers_for_well(well_df, model, scaler, config, device):
    model.eval()
    patch_height = config['finetuning']['model_params']['patch_height']
    # Use only the feature columns the scaler was trained on
    feature_cols = scaler.feature_names_in_
    well_numeric = well_df[feature_cols].fillna(0)
    scaled_data = scaler.transform(well_numeric).astype(np.float32)
    predicted_layers = []
    with torch.no_grad():
        for i in range(0, len(scaled_data) - patch_height + 1, patch_height):
            patch = scaled_data[i:i + patch_height]
            patch_tensor = torch.from_numpy(patch).unsqueeze(0).unsqueeze(0).to(device)
            outputs = model(patch_tensor)
            probs, loc_info = outputs[0, :, 0].sigmoid(), outputs[0, :, 1:]
            keep = probs > config['inference']['prediction_score_threshold']
            for top_rel, height_rel in loc_info[keep].cpu().numpy():
                # Get the absolute depth from the original dataframe's index
                start_depth = well_df.index[i]
                abs_top = start_depth + (top_rel * patch_height * (well_df.index[1] - well_df.index[0]))
                abs_h = height_rel * patch_height * (well_df.index[1] - well_df.index[0])
                predicted_layers.append({'Top': abs_top, 'bottom': abs_top + abs_h, 'Height': abs_h})
    predicted_layers.sort(key=lambda x: x['Top'])
    return predicted_layers

def plot_predicted_layers(well1_name, well2_name, layers1, layers2, path):
    plt.style.use('ggplot')
    fig, ax = plt.subplots(figsize=(10, 12))
    if not layers1 and not layers2:
      print("Cannot plot, no layers were predicted for either well.")
      return None

    max_depth1 = layers1[-1]['bottom'] if layers1 else 0
    max_depth2 = layers2[-1]['bottom'] if layers2 else 0
    max_depth = max(max_depth1, max_depth2)

    ax.set_ylim(max_depth + 50, -50); ax.set_xlim(-0.5, 2.5)
    for l in layers1: ax.add_patch(patches.Rectangle((0, l['Top']), 1, l['Height'], ec='k', fc='cyan', alpha=0.6))
    for l in layers2: ax.add_patch(patches.Rectangle((1.5, l['Top']), 1, l['Height'], ec='k', fc='orange', alpha=0.6))
    ax.set_xticks([0.5, 2]); ax.set_xticklabels([well1_name, well2_name], fontsize=14)
    ax.set_ylabel('Depth', fontsize=12); ax.set_title('Well to Well - Predicted Layers', fontsize=16)
    plt.savefig(path)
    plt.show()
    return wandb.Image(path)

print("✅ Inference functions defined.")

✅ Inference functions defined.


In [16]:
#@title Step 6: Run Inference
# --- ACTION REQUIRED: Define the well pairs you want to plot ---
# Use the well filenames from your folder, WITHOUT the .las extension
well_pairs_to_plot = [
    ("15_9-13", "15_9-15"),
    # e.g., ("WELL_FILENAME_A", "WELL_FILENAME_B"),
]
# -----------------------------------------------------------------

# --- Load Model and Artifacts ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
try:
    model = W2WTransformerModel(config).to(device)
    model.load_state_dict(torch.load(config['paths']['final_model_path'], map_location=device))
    scaler = load(config['paths']['std_scaler_path'])
    print("✅ Model and scaler loaded successfully.")
except Exception as e:
    print(f"🛑 FATAL ERROR loading files: {e}")
    model = None

# --- Run Inference Loop ---
if model:
    wandb.login()
    with wandb.init(project=config['wandb']['project'], entity=config['wandb'].get('entity'), job_type='inference-run-las') as run:
        plots_to_log = {}
        for i, (well1_name, well2_name) in enumerate(well_pairs_to_plot):
            print(f"\n--- Processing Pair {i+1}: {well1_name} vs {well2_name} ---")
            try:
                # Construct file paths and load .las files
                las1_path = os.path.join(config['paths']['raw_las_folder'], f"{well1_name}.las")
                las2_path = os.path.join(config['paths']['raw_las_folder'], f"{well2_name}.las")

                well1_df = lasio.read(las1_path).df()
                well2_df = lasio.read(las2_path).df()

                layers1 = predict_layers_for_well(well1_df, model, scaler, config, device)
                layers2 = predict_layers_for_well(well2_df, model, scaler, config, device)
                print(f"--> Found {len(layers1)} layers in {well1_name} and {len(layers2)} in {well2_name}")

                output_path = f"predicted_{well1_name}_vs_{well2_name}.png"
                img = plot_predicted_layers(well1_name, well2_name, layers1, layers2, output_path)
                if img: plots_to_log[f"Plot_{i+1}_{well1_name}"] = img

            except FileNotFoundError as fe:
                print(f"ERROR: Could not find .las file. {fe}")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")

        if plots_to_log:
            print("\n--> Logging all plots to Weights & Biases...")
            wandb.log(plots_to_log)
            print("✅ Inference complete. Check your W&B dashboard.")

Using device: cpu
🛑 FATAL ERROR loading files: [Errno 2] No such file or directory: '/content/trained_models/boundary_detector/final_model.pt'
